In [1]:
import sys
import os
from pathlib import Path

# Add parent directory to Python path
notebook_dir = Path(os.getcwd())
project_dir = notebook_dir.parent
sys.path.append(str(project_dir))


# notebooks/02_model_training.ipynb

import pandas as pd
import numpy as np
from src.data_processor import DataProcessor
from src.models.expenditure_predictor import ExpenditurePredictor
import plotly.graph_objects as go

# Prepare data
processor = DataProcessor()
df = processor.load_household_data('../data/sample_household_data.csv')

# Train separate models for each household
household_models = {}
household_metrics = {}

for household in df['HouseholdID'].unique():
    print(f"\nTraining model for {household}")
    
    # Filter data for current household
    household_df = df[df['HouseholdID'] == household].copy()
    X, y = processor.prepare_training_data(household_df)
    
    # Split data
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    # Train model
    predictor = ExpenditurePredictor()
    history = predictor.train(X_train, y_train)
    
    # Evaluate model
    y_pred = predictor.predict(X_test)
    mse = np.mean((y_test - y_pred.flatten()) ** 2)
    
    # Store model and metrics
    household_models[household] = predictor
    household_metrics[household] = {
        'mse': mse,
        'history': history
    }
    
    print(f"MSE for {household}: {mse:.4f}")


# Save models for each household
for household in household_models:
    model_path = f'../src/models/saved/model_{household}.keras'
    household_models[household].save(model_path)
    print(f"Saved model for {household} at {model_path}")

# Visualize training history for each household
fig = go.Figure()
for household, metrics in household_metrics.items():
    history = metrics['history']
    fig.add_trace(go.Scatter(
        y=history.history['loss'],
        name=f'{household} Training Loss',
        mode='lines'
    ))
    fig.add_trace(go.Scatter(
        y=history.history['val_loss'],
        name=f'{household} Validation Loss',
        mode='lines',
        line=dict(dash='dash')
    ))

fig.update_layout(
    title='Training History by Household',
    xaxis_title='Epoch',
    yaxis_title='Loss',
    hovermode='x unified'
)
fig.show()

# Compare predictions vs actual for each household
for household in household_models:
    household_df = df[df['HouseholdID'] == household].copy()
    X, y = processor.prepare_training_data(household_df)
    
    # Get predictions
    y_pred = household_models[household].predict(X)
    
    # Plot actual vs predicted
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        y=y,
        name='Actual',
        mode='lines'
    ))
    fig.add_trace(go.Scatter(
        y=y_pred.flatten(),
        name='Predicted',
        mode='lines'
    ))
    
    fig.update_layout(
        title=f'Actual vs Predicted Expenditure - {household}',
        xaxis_title='Time',
        yaxis_title='Expenditure ($)',
        hovermode='x unified'
    )
    fig.show()



Training model for H1
Epoch 1/100
3/3 [==============================] - 4s 365ms/step - loss: 0.3391 - val_loss: 0.1822
Epoch 2/100
3/3 [==============================] - 0s 15ms/step - loss: 0.2750 - val_loss: 0.1575
Epoch 3/100
3/3 [==============================] - 0s 14ms/step - loss: 0.2388 - val_loss: 0.1337
Epoch 4/100
3/3 [==============================] - 0s 14ms/step - loss: 0.2060 - val_loss: 0.1076
Epoch 5/100
3/3 [==============================] - 0s 15ms/step - loss: 0.1701 - val_loss: 0.0800
Epoch 6/100
3/3 [==============================] - 0s 14ms/step - loss: 0.1318 - val_loss: 0.0520
Epoch 7/100
3/3 [==============================] - 0s 13ms/step - loss: 0.0916 - val_loss: 0.0265
Epoch 8/100
3/3 [==============================] - 0s 13ms/step - loss: 0.0516 - val_loss: 0.0098
Epoch 9/100
3/3 [==============================] - 0s 16ms/step - loss: 0.0222 - val_loss: 0.0103
Epoch 10/100
3/3 [==============================] - 0s 15ms/step - loss: 0.0104 - val_loss: 0.

4/4 [==============================] - 0s 2ms/step


4/4 [==============================] - 0s 2ms/step


4/4 [==============================] - 0s 2ms/step


4/4 [==============================] - 0s 1ms/step


4/4 [==============================] - 0s 2ms/step
